In [32]:
import copy

In [57]:
class Stp:
    def __init__(self,filename):
        self.BRIDGES=[]
        self.LANS=[]
        self.B_ADJ=dict()
        self.L_B_ADJ=dict()
        self.filename=filename

    def readInput(self):
        file = open(self.filename,"r")
        mode=int(file.readline())
        n=int(file.readline())
        for i in range(n):
            line=file.readline()
            name=line.split(':')[0]
            lans=[x.rstrip() for x in line.split(':')[1] if x.isalpha()]
            mess=Message(name,0,name,name)
            self.BRIDGES.append(Bridge(name,lans,mess))

    def fillLans(self):
        lannames=[]
        for bridge in self.BRIDGES:
            for lans in bridge.lans:
                if lans not in lannames:
                    lannames.append(lans)
                    self.LANS.append(Lan(lans,[]))
        for lans in self.LANS:
            for bridge in self.BRIDGES:
                if lans.name in bridge.lans:
                    lans.bridges.append(bridge.name)

    def initialize(self):
        self.BRIDGES=[]
        self.LANS=[]
        self.readInput()
        self.fillLans()
        for bridge in self.BRIDGES:
            for lan in self.LANS:
                if lan.name in bridge.lans:
                    for bridge_inner in self.BRIDGES:
                        if bridge_inner.name in lan.bridges and bridge_inner.name!=bridge.name and bridge_inner.name not in bridge.adj:
                            bridge.adj.append(bridge_inner.name)
               
            #print()


    def displayMessage(self,time,type,node,message):
        print('{} {} {}'.format(time,type,node.name),end=' ')
        message.display()
                        
    def sendMessages(self, time):
        for bridge in self.BRIDGES: 
            for i in bridge.lans:
                for lan in self.LANS:
                    if lan.name==i:
                        for j in lan.bridges:
                            if j!=bridge.name:
                                message_temp=copy.deepcopy(bridge.message)
                                message_temp.sender=bridge.name
                                message_temp.destination=j
                                lan.messages.append(message_temp)
            
            for i in bridge.lans:
                self.displayMessage(time,'s',bridge,message_temp)
    
    def receiveMessages(self, time):
        for bridge in self.BRIDGES:
            for lan in self.LANS:
                if bridge.name in lan.bridges:
                    for message in lan.messages:                
                        if message.destination==bridge.name:
                            self.displayMessage(time,'r',bridge,message)
                            bridge.received.append(message)
        for lan in self.LANS:
            lan.messages=[]

    def updateMessages(self):
        flag=False
        for bridge in self.BRIDGES:
            for received in bridge.received:
                #received.display()
                received_copy=copy.deepcopy(received)
                received_copy.distance+=1                
                message_temp=bridge.message.compare(received_copy)
                flag=flag or bridge.message.compareBool(message_temp)
                bridge.message=message_temp
                #bridge.message.display()
            #print()
            bridge.received=[]
        return flag
    
    def establishConnection(self): 
        time=0
        flag=True
        while True:
            self.sendMessages(time)
            self.receiveMessages(time+1)
            flag=self.updateMessages()
            if flag == False:
                break
            time+=1
            

In [58]:
class Bridge:
    def __init__(self,name,lans,message):
        self.name=name
        self.lans=lans
        self.message=message
        self.adj=[]
        self.received=[]

    def display(self):
        print('{}: {}: {}'.format(self.name, self.lans, self.adj))
        self.message.display()

In [59]:
class Lan:
    def __init__(self,name,bridges):
        self.name=name
        self.bridges=bridges
        self.messages=[]

    def display(self):
        print('{}: {}'.format(self.name, self.bridges))

In [60]:
class Message:
    def __init__(self,root,distance,sender,destination):
        self.root=root
        self.distance=distance
        self.sender=sender
        self.destination=destination

    def compare(self, message):
        if message.root<self.root:
            return message
        elif message.distance<self.distance:
            return message
        elif message.sender<self.sender:
            return message
        else:
            return self
        
    def compareBool(self, message):
        if message.root<self.root:
            return True
        elif message.distance<self.distance:
            return True
        elif message.sender<self.sender:
            return True
        else:
            return False  
        

    def display(self):
        print('({} {} {})'.format(self.root, self.distance, self.sender))

In [61]:
if __name__=='__main__':
    stpobj=Stp(r"input\input1.txt")
    stpobj.initialize()
    for bridge in stpobj.BRIDGES:
        bridge.display()
    for lan in stpobj.LANS:
            lan.display()
    stpobj.establishConnection()
    for bridge in stpobj.BRIDGES:
            bridge.display()

B1: ['A', 'G', 'B']: ['B2', 'B3']
(B1 0 B1)
B2: ['G', 'F']: ['B1', 'B4']
(B2 0 B2)
B3: ['B', 'C']: ['B1', 'B4', 'B5']
(B3 0 B3)
B4: ['C', 'F', 'E']: ['B2', 'B3', 'B5']
(B4 0 B4)
B5: ['C', 'D', 'E']: ['B3', 'B4']
(B5 0 B5)
A: ['B1']
G: ['B1', 'B2']
B: ['B1', 'B3']
F: ['B2', 'B4']
C: ['B3', 'B4', 'B5']
E: ['B4', 'B5']
D: ['B5']
0 s B1 (B1 0 B1)
0 s B1 (B1 0 B1)
0 s B1 (B1 0 B1)
0 s B2 (B2 0 B2)
0 s B2 (B2 0 B2)
0 s B3 (B3 0 B3)
0 s B3 (B3 0 B3)
0 s B4 (B4 0 B4)
0 s B4 (B4 0 B4)
0 s B4 (B4 0 B4)
0 s B5 (B5 0 B5)
0 s B5 (B5 0 B5)
0 s B5 (B5 0 B5)
1 r B1 (B2 0 B2)
1 r B1 (B3 0 B3)
1 r B2 (B1 0 B1)
1 r B2 (B4 0 B4)
1 r B3 (B1 0 B1)
1 r B3 (B4 0 B4)
1 r B3 (B5 0 B5)
1 r B4 (B2 0 B2)
1 r B4 (B3 0 B3)
1 r B4 (B5 0 B5)
1 r B4 (B5 0 B5)
1 r B5 (B3 0 B3)
1 r B5 (B4 0 B4)
1 r B5 (B4 0 B4)
1 s B1 (B1 0 B1)
1 s B1 (B1 0 B1)
1 s B1 (B1 0 B1)
1 s B2 (B1 1 B2)
1 s B2 (B1 1 B2)
1 s B3 (B1 1 B3)
1 s B3 (B1 1 B3)
1 s B4 (B2 1 B4)
1 s B4 (B2 1 B4)
1 s B4 (B2 1 B4)
1 s B5 (B3 1 B5)
1 s B5 (B3 1 B5)
1 s B5 (B